In [2]:
%cd drive/MyDrive/Complexity-Measure/

/content/drive/MyDrive/Complexity-Measure


In [3]:
!pip install gower

In [4]:
from tensorflow.keras.applications import VGG16, Xception, ResNet50V2
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
import numpy as np
import cv2
from tensorflow.keras.datasets import mnist
from measures import *
import math

iris dataset train accuracy: 0.9925925925925926
iris dataset test accuracy: 0.9333333333333333
-----------------------------------------------


In [5]:
from keras import backend as K
K.clear_session()

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train.shape, y_train.shape

((60000, 28, 28), (60000,))

In [15]:
def one_to_three_dim(x_train, dim):
  img_lst = []
  for i in range(len(x_train)):
    img = cv2.resize(x_train[i], dim, interpolation = cv2.INTER_AREA) 
    img = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    img_lst.append(img)
  rgb_arr = np.stack([img_lst],axis=4)
  rgb_arr_to_3d = np.squeeze(rgb_arr, axis=4)
  return rgb_arr_to_3d

In [16]:
def feature_extractor(images, model, x, y, z):
  feature_list = []
  for i in range(len(images)):
    img_array = np.expand_dims(images[i], axis=0)
    features = model.predict(img_array)
    feature_list.append(features)
  feat_lt = np.reshape(feature_list, (-1, x*y*z))
  return feat_lt

In [17]:
dim = (72, 72)
rgb_list = one_to_three_dim(x_train, dim)
print(rgb_list.shape)

(60000, 72, 72, 3)


In [18]:
vgg = VGG16(weights='imagenet',
            include_top=False,
            input_shape=(72, 72, 3))

for layers in vgg.layers:
  layers.trainable=False

print(vgg.output)


58892288/58889256 [==============================] - 1s 0us/step
KerasTensor(type_spec=TensorSpec(shape=(None, 2, 2, 512), dtype=tf.float32, name=None), name='block5_pool/MaxPool:0', description="created by layer 'block5_pool'")


In [19]:
vgg_features = feature_extractor(rgb_list[0:1000, :, :, :], vgg, 2, 2, 512)
np.save("vgg16_features.npy", vgg_features)

In [14]:
xception = Xception(weights='imagenet',
            include_top=False,
            input_shape=(72, 72, 3))

for layers in xception.layers:
  layers.trainable=False

print(xception.output)

83689472/83683744 [==============================] - 1s 0us/step
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 3, 2048), dtype=tf.float32, name=None), name='block14_sepconv2_act/Relu:0', description="created by layer 'block14_sepconv2_act'")


In [23]:
xception_features = feature_extractor(rgb_list[0:1000, :, :, :], xception, 3, 3, 2048)
np.save("xception_features.npy", xception_features)

In [15]:
resnet = ResNet50V2(weights='imagenet',
            include_top=False,
            input_shape=(72, 72, 3))

for layers in resnet.layers:
  layers.trainable=False

print(resnet.output)

94674944/94668760 [==============================] - 1s 0us/step
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 3, 2048), dtype=tf.float32, name=None), name='post_relu/Relu:0', description="created by layer 'post_relu'")


In [24]:
resnet_features = feature_extractor(rgb_list[0:1000, :, :, :], resnet, 3, 3, 2048)
np.save("resnet_features.npy", resnet_features)

In [25]:
vgg_features.shape, xception_features.shape, resnet_features.shape

((1000, 2048), (1000, 18432), (1000, 18432))

In [3]:
!pip install gower

  Created wheel for gower: filename=gower-0.0.5-py3-none-any.whl size=4231 sha256=78c0cceb6ff0498ef63ea095194bb555bc0a40e41d14d43d2f9d687b31f068aa
  Stored in directory: /root/.cache/pip/wheels/3e/f9/9a/67122a959a424e9cbb4557a8366c871a30e31cd75f0d003db4
Successfully built gower


In [7]:
X_mnist = np.load("vgg16_features.npy")
Y_mnist = y_train[0:1000]

In [8]:
precompute_fx_mnist = precompute_fx(X_mnist, Y_mnist)
cls_index_mnist = precompute_fx_mnist['cls_index']
cls_n_ex_mnist = precompute_fx_mnist['cls_n_ex']
ovo_comb_mnist = precompute_fx_mnist['ovo_comb']
# prepcomp_vals_mnist = {}
# classes, class_freqs = np.unique(Y_mnist, return_counts=True)
# cls_index_mnist = [np.equal(Y_mnist, i) for i in range(classes.shape[0])]
# #cls_n_ex = np.array([np.sum(aux) for aux in cls_index])
# cls_n_ex_mnist = list(class_freqs)
# ovo_comb_mnist = list(itertools.combinations(range(classes.shape[0]), 2))
# prepcomp_vals_mnist["ovo_comb"] = ovo_comb_mnist
# prepcomp_vals_mnist["cls_index"] = cls_index_mnist
# prepcomp_vals_mnist["cls_n_ex"] = cls_n_ex_mnist

In [7]:
ft_F1(X_mnist, cls_index_mnist,cls_n_ex_mnist)

0.36221810726716736

In [8]:
ft_F1v(X_mnist, ovo_comb_mnist, cls_index_mnist)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [7]:
ft_F2(X_mnist, ovo_comb_mnist, cls_index_mnist)

/content/drive/MyDrive/Complexity-Measure/measures.py:333: RuntimeWarning: invalid value encountered in true_divide
  ratio = overlap_/range_


0.0

In [8]:
ft_F3(X_mnist, ovo_comb_mnist, cls_index_mnist, cls_n_ex_mnist)

0.0

In [19]:
ft_F4(X_mnist, ovo_comb_mnist, cls_index_mnist, cls_n_ex_mnist)
# ft_F4(X_iris, ovo_comb_iris, cls_index_iris, cls_n_ex_iris)

0.043333333333333335

In [9]:
from sklearn.svm import SVC
model_mnist = SVC(kernel = 'linear')
model_mnist.fit(X_mnist, Y_mnist)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [10]:
ft_R1(model_mnist, X_mnist, Y_mnist)

0.0

In [11]:
ft_R2(model_mnist, X_mnist, Y_mnist)

0.0

In [12]:
ft_R3(model_mnist, X_mnist, cls_index_mnist, cls_n_ex_mnist)

0.0

In [13]:
ft_N1(X_mnist, Y_mnist)

0.187

In [15]:
ft_N2(X_mnist, Y_mnist, cls_index_mnist)

0.4569347842733395

In [16]:
ft_N3(X_mnist, Y_mnist)

0.10899999999999999

In [17]:
ft_N4(X_mnist, Y_mnist, cls_index_mnist)

0.0050000000000000044

In [18]:
ft_T1(X_mnist, Y_mnist, cls_index_mnist, 0.001)

KeyboardInterrupt: ignored

In [19]:
LSC(X_mnist, Y_mnist, cls_index_mnist)

0.990399

In [20]:
ft_Density(X_mnist, Y_mnist)

0.900002002002002

In [21]:
ft_ClsCoef(X_mnist, Y_mnist)

0.9010702312042197

In [22]:
ft_Hubs(X_mnist, Y_mnist, 6)

0.9743793037518604

In [23]:
precomp_pca = precompute_pca_tx(X_mnist)
mnist_m = precomp_pca['m']
mnist_n = precomp_pca['n']
mnist_m_ = precomp_pca['m_']

In [24]:
ft_T2(mnist_m, mnist_n)

2.048

In [25]:
ft_T3(mnist_m, mnist_m_)

4.266666666666667

In [26]:
ft_T4(mnist_m, mnist_n)

0.48828125

In [27]:
ft_C1(cls_n_ex_mnist)

0.001914079013448755

In [28]:
ft_C2(cls_n_ex_mnist)

0.001115691269140795